<a href="https://colab.research.google.com/github/MashKeyZ/movie-reviews-classifier/blob/main/houseing_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt

In [5]:
(x_train, y_train), (x_test, y_test)= keras.datasets.imdb.load_data(num_words=10000)


In [8]:
word_index = keras.datasets.imdb.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()} #Adding 3 to each value, to open a space for the tags


In [9]:
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"]=2
word_index["<UNUSED>"]=3

In [15]:
#reverse the words so that the coded values can point to the words, swap the values with the keys
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])


decode training data to human readable words

In [ ]:
def decode_review(text):
  return " ".join([reverse_word_index.get(i,"?") for i in text])

The data must be shaped for the input nuerons. The sentences will have different lenghts and we have to standardize the length of each setence using the <PAD>padding tag

In [16]:
x_train = keras.preprocessing.sequence.pad_sequences(x_train,value=word_index["<PAD>"],padding="post",maxlen=250)
x_test = keras.preprocessing.sequence.pad_sequences(x_test,value=word_index["<PAD>"],padding="post",maxlen=250)

**Create Model

In [17]:
model = keras.Sequential()
model.add(keras.layers.Embedding(10000,16)) # Embedding layer to group words with similar meanings together
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16,activation ="relu"))
model.add(keras.layers.Dense(1,activation="sigmoid"))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160289 (626.13 KB)
Trainable params: 160289 (626.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy", #Since we are having 2 possible values
    metrics=["accuracy"]
)

split data for training and validation

In [22]:
y_val = y_train[:10000]
y_train = y_train[10000:]

x_val = x_train[:10000]
x_train = x_train[10000:]

In [26]:
fitModel = model.fit(
    x_train,y_train,epochs = 10, batch_size = 512, validation_data = (x_val,y_val),verbose=1
)

Epoch 1/10
30/30 [==============================] - 1s 47ms/step - loss: 0.6156 - accuracy: 0.7797 - val_loss: 0.6041 - val_accuracy: 0.7871
Epoch 2/10
30/30 [==============================] - 1s 31ms/step - loss: 0.5748 - accuracy: 0.8102 - val_loss: 0.5637 - val_accuracy: 0.8016
Epoch 3/10
30/30 [==============================] - 1s 39ms/step - loss: 0.5296 - accuracy: 0.8273 - val_loss: 0.5226 - val_accuracy: 0.8166
Epoch 4/10
30/30 [==============================] - 1s 43ms/step - loss: 0.4836 - accuracy: 0.8447 - val_loss: 0.4828 - val_accuracy: 0.8307
Epoch 5/10
30/30 [==============================] - 1s 21ms/step - loss: 0.4409 - accuracy: 0.8604 - val_loss: 0.4460 - val_accuracy: 0.8419
Epoch 6/10
30/30 [==============================] - 1s 25ms/step - loss: 0.4028 - accuracy: 0.8724 - val_loss: 0.4162 - val_accuracy: 0.8502
Epoch 7/10
30/30 [==============================] - 1s 22ms/step - loss: 0.3709 - accuracy: 0.8796 - val_loss: 0.3908 - val_accuracy: 0.8567
Epoch 8/10
30

In [27]:
acc = model.evaluate(x_test,y_test)

print("Model accuracy : ",acc)

782/782 [==============================] - 2s 3ms/step - loss: 0.3519 - accuracy: 0.8612
Model accuracy :  [0.3519497215747833, 0.8611999750137329]
